In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

def IntegerSafe(value): # In case there are non-integer type to be converted.
    try:
        return int(value)
    except ValueError:
        return None

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()


business = sc.textFile("../Data/SF_business/filtered_registered_business_sf.csv")\
             .map(lambda x : x.split(','))\
             .map(lambda x : (IntegerSafe(x[0]), x[1], x[2], x[3], x[4]))

supervisor = sc.textFile("../Data/SF_business/supervisor_sf.csv")\
               .map(lambda x : x.split(","))\
               .map(lambda x: (IntegerSafe(x[0]), IntegerSafe(x[1])))
              

business_schema = StructType([ StructField("zip", IntegerType(), True),
                               StructField("name", StringType(), False),
                               StructField("street", StringType(), True),
                               StructField("city", StringType(), True),
                               StructField("state", StringType(), True)
                            ])

supervisor_schema = StructType([ StructField("zip", IntegerType(), False),
                    StructField("id", IntegerType(), False)
                    ])

business_df = ss.createDataFrame(business, business_schema)
supervisor_df = ss.createDataFrame(supervisor, supervisor_schema)

## inner joins

In [2]:
business_df.join(supervisor_df, 'zip').show(5)

+-----+--------------------+--------------------+-------------+-----+---+
|  zip|                name|              street|         city|state| id|
+-----+--------------------+--------------------+-------------+-----+---+
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  2|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  6|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  3|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  5|
|94109|Stephens Institut...|        1055 Pine St|San Francisco|   CA|  2|
+-----+--------------------+--------------------+-------------+-----+---+
only showing top 5 rows



In [3]:
business_df.join(supervisor_df, business_df['zip'] == supervisor_df['zip']).show(5)

+-----+--------------------+--------------------+-------------+-----+-----+---+
|  zip|                name|              street|         city|state|  zip| id|
+-----+--------------------+--------------------+-------------+-----+-----+---+
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|94109|  2|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|94109|  6|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|94109|  3|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|94109|  5|
|94109|Stephens Institut...|        1055 Pine St|San Francisco|   CA|94109|  2|
+-----+--------------------+--------------------+-------------+-----+-----+---+
only showing top 5 rows



In [4]:
business_df.join(supervisor_df, 'zip', 'inner').show(5)

+-----+--------------------+--------------------+-------------+-----+---+
|  zip|                name|              street|         city|state| id|
+-----+--------------------+--------------------+-------------+-----+---+
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  2|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  6|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  3|
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  5|
|94109|Stephens Institut...|        1055 Pine St|San Francisco|   CA|  2|
+-----+--------------------+--------------------+-------------+-----+---+
only showing top 5 rows



## outer joins

In [5]:
business_df.join(supervisor_df, 'zip', 'left_outer').show(5)

+-----+--------------------+-------------------+-----------+-----+----+
|  zip|                name|             street|       city|state|  id|
+-----+--------------------+-------------------+-----------+-----+----+
| 2142|        Trecler Alan|        1 Rogers St|  Cambridge|   MA|null|
| 2142|Aegerion Pharmace...|        101 Main St|  Cambridge|   MA|null|
| 8086|         Aramsco Inc| 1480 Grandview Ave|  Thorofare|   NJ|null|
|18051|Five Thousand For...|       8020 Mine St|Fogelsville|   PA|null|
|21220|Social Solutions ...|425 Williams Ct 100|  Baltimore|   MD|null|
+-----+--------------------+-------------------+-----------+-----+----+
only showing top 5 rows



In [6]:
business_df.join(supervisor_df, 'zip', 'right_outer').show(5)

+-----+--------------------+--------------------+-------------+-----+---+
|  zip|                name|              street|         city|state| id|
+-----+--------------------+--------------------+-------------+-----+---+
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|  2|
|94109|Stephens Institut...|        1055 Pine St|San Francisco|   CA|  2|
|94109|     Alioto F Co Inc|    440 Jefferson St|San Francisco|   CA|  2|
|94109|     Haines Robert D|   786-792 Sutter St|San Francisco|   CA|  2|
|94109|Avis Rent A Car S...|         675 Post St|San Francisco|   CA|  2|
+-----+--------------------+--------------------+-------------+-----+---+
only showing top 5 rows



## leftsemi join

In [7]:
business_df.join(supervisor_df, 'zip', 'leftsemi').show(5)

+-----+--------------------+--------------------+-------------+-----+
|  zip|                name|              street|         city|state|
+-----+--------------------+--------------------+-------------+-----+
|94109|Stephens Institut...|1835-49 Van Ness Ave|San Francisco|   CA|
|94109|Stephens Institut...|        1055 Pine St|San Francisco|   CA|
|94109|     Alioto F Co Inc|    440 Jefferson St|San Francisco|   CA|
|94109|     Haines Robert D|   786-792 Sutter St|San Francisco|   CA|
|94109|Avis Rent A Car S...|         675 Post St|San Francisco|   CA|
+-----+--------------------+--------------------+-------------+-----+
only showing top 5 rows

